In [ ]:
%pip install rdflib

In [1]:
from rdflib import Graph
g = Graph()
g.parse('https://datos.madrid.es/egob/catalogo.rdf')

https://datos.madrid.es/egob/catalogo/300707-0-satisfacción, música, danza does not look like a valid URI, trying to serialize this will break.
https://datos.madrid.es/egob/catalogo/300706-0-participación ciudadana does not look like a valid URI, trying to serialize this will break.
https://datos.madrid.es/egob/catalogo/300681-0-adicciones, CAD, tratamiento does not look like a valid URI, trying to serialize this will break.
https://datos.madrid.es/egob/catalogo/300659-0-EMT-encuesta-bus rapid does not look like a valid URI, trying to serialize this will break.
https://datos.madrid.es/egob/catalogo/300649-0-viveros municipales-variedades does not look like a valid URI, trying to serialize this will break.
https://datos.madrid.es/egob/catalogo/300643-0-estudio-centros dia-infantiles does not look like a valid URI, trying to serialize this will break.
https://datos.madrid.es/egob/catalogo/300633-0-estudio-educación social does not look like a valid URI, trying to serialize this will brea

<Graph identifier=Nee36f74bf8754d199ff41e7ad2583967 (<class 'rdflib.graph.Graph'>)>

In [ ]:
i = 0
for s, p, o in g:
    print(i, s, p, o, sep="; ")
    i += 1

In [ ]:
from rdflib.namespace import DCAT

In [2]:
knows_query = """
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
SELECT *
WHERE {
    ?n dct:title ?d1 .
    ?n dcat:distribution ?d2 .
    ?d2 dcat:accessURL ?d3 .
    ?d2 dcat:mediaType ?d4 .
    FILTER (?d4 = "text/csv")
}"""

qres = g.query(knows_query)
for row in qres:
    print(row.n ,row.d1, row.d2, row.d3, row.d4, sep = "; ")

https://datos.madrid.es/egob/catalogo/300708-0-encuesta-laboratorio-salud; Encuesta de satisfacción realizada entre las personas que utilizan los servicios analíticos del Laboratorio Municipal de Salud Pública 2023; Nf6d2bf26fd1b45db856093ec0c820623; https://datos.madrid.es/egob/catalogo/300708-0-encuesta-laboratorio-salud.csv; text/csv
https://datos.madrid.es/egob/catalogo/300708-0-encuesta-laboratorio-salud; Encuesta de satisfacción realizada entre las personas que utilizan los servicios analíticos del Laboratorio Municipal de Salud Pública 2023; Ne9694c727fa54f938cd7c504da8a620a; https://datos.madrid.es/egob/catalogo/300708-2-encuesta-laboratorio-salud.csv; text/csv
https://datos.madrid.es/egob/catalogo/300706-0-participación ciudadana; Miembros y colaboradores de la comunidad The Participatory Group; N6a7b31537d3c4de6a7816b36a5da0e12; https://datos.madrid.es/egob/catalogo/300706-0-participación ciudadana.csv; text/csv
https://datos.madrid.es/egob/catalogo/300702-0-satisfacción-arch

In [ ]:
knows_query = """
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
SELECT *
WHERE {
    ?n dct:title ?d1 .
    ?n dcat:distribution ?d2 .
    ?d2 dcat:accessURL ?d3 .
    ?d2 dct:title ?d5 .
    ?d2 dcat:byteSize ?d6 .
    ?d2 dcat:mediaType ?d4 .
    FILTER (?d4 = "text/csv")
}"""

qres = g.query(knows_query)
for row in qres:
    print(row.n, row.d1,  row.d3, row.d4, row.d5, row.d6, sep="; ")

In [ ]:
%pip install polars

In [4]:
from datetime import datetime

knows_query = """
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
SELECT *
WHERE {
    ?n dct:title ?d1 .
    ?n dcat:distribution ?d2 .
    ?n dct:issued ?issued .
    ?n dct:modified ?modified .
    ?d2 dcat:accessURL ?d3 .
    ?d2 dct:title ?d5 .
    ?d2 dcat:byteSize ?d6 .
    ?d2 dcat:mediaType ?d4 .
    ?n dcat:theme ?d7 .
    FILTER (?d4 = "text/csv")
}"""

qres = g.query(knows_query)

results = (
    {'dataset': row.n,
     'title': row.d1,
     'accessURL': row.d3,
     'mediaType': row.d4,
     'dist_title': row.d5,
     'byteSize': int(row.d6),
     'issued': datetime.fromisoformat(row.issued), 'modified': datetime.fromisoformat(row.modified)
     ,'theme': row.d7
     }
    for row in qres
)

In [5]:
import polars as pl

dfp = pl.DataFrame(results)

dfp

dataset,title,accessURL,mediaType,dist_title,byteSize,issued,modified,theme
str,str,str,str,str,i64,datetime[μs],datetime[μs],str
"""https://datos.madrid.es/egob/c…","""Encuesta de satisfacción reali…","""https://datos.madrid.es/egob/c…","""text/csv""","""Encuesta 2023. Resultados de l…",17408,2024-10-17 13:39:57,2024-10-17 13:40:39,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Encuesta de satisfacción reali…","""https://datos.madrid.es/egob/c…","""text/csv""","""Encuesta 2023. Diseño de regis…",1024,2024-10-17 13:39:57,2024-10-17 13:40:39,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Miembros y colaboradores de la…","""https://datos.madrid.es/egob/c…","""text/csv""","""""The Participatory Group""""",2048,2024-09-09 14:53:32,2024-09-09 14:53:40,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Estudio de satisfacción de las…","""https://datos.madrid.es/egob/c…","""text/csv""","""2023""",25600,2024-08-07 16:46:38,2024-10-14 08:23:01,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Ofertas de Empleo Público del …","""https://datos.madrid.es/egob/c…","""text/csv""","""Oferta de empleo público 2018-…",93184,2024-08-05 14:16:00,2024-08-05 14:16:35,"""http://datos.gob.es/kos/sector…"
…,…,…,…,…,…,…,…,…
"""https://datos.madrid.es/egob/c…","""Urgencias atendidas desde los …","""https://datos.madrid.es/egob/c…","""text/csv""","""2018""",905216,2019-01-22 15:24:22,2024-10-03 15:05:57,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Viviendas en alquiler adjudica…","""https://datos.madrid.es/egob/c…","""text/csv""","""Viviendas en alquiler adjudica…",1024,2017-04-28 15:50:07,2023-07-07 10:10:32,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Viviendas promocionadas y reha…","""https://datos.madrid.es/egob/c…","""text/csv""","""Promociones y rehabilitaciones…",39936,2021-07-14 16:06:47,2023-07-07 10:54:28,"""http://datos.gob.es/kos/sector…"


In [9]:
display(dfp.sort(by='modified', descending=True).head())

display(dfp.shape)

dataset,title,accessURL,mediaType,dist_title,byteSize,issued,modified,theme
str,str,str,str,str,i64,datetime[μs],datetime[μs],str
"""https://datos.madrid.es/egob/c…","""Panel de Hogares de la ciudad …","""https://datos.madrid.es/egob/c…","""text/csv""","""2023. Información general""",2305024,2022-06-17 10:19:06,2024-12-03 16:07:42,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Panel de Hogares de la ciudad …","""https://datos.madrid.es/egob/c…","""text/csv""","""2023. Miembros""",3249152,2022-06-17 10:19:06,2024-12-03 16:07:42,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Panel de Hogares de la ciudad …","""https://datos.madrid.es/egob/c…","""text/csv""","""2021. Información general""",3418112,2022-06-17 10:19:06,2024-12-03 16:07:42,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Panel de Hogares de la ciudad …","""https://datos.madrid.es/egob/c…","""text/csv""","""2021. Miembros""",4260864,2022-06-17 10:19:06,2024-12-03 16:07:42,"""http://datos.gob.es/kos/sector…"
"""https://datos.madrid.es/egob/c…","""Huella normativa""","""https://datos.madrid.es/egob/c…","""text/csv""","""Huella normativa""",80896,2021-02-25 08:21:21,2024-12-03 14:32:00,"""http://datos.gob.es/kos/sector…"


(3458, 9)

In [18]:
with pl.Config(fmt_str_lengths=1000, tbl_width_chars=1000):
    display(dfp.select('theme').group_by('theme').len())


theme,len
str,u32
"""http://datos.gob.es/kos/sector-publico/sector/turismo""",1
"""http://datos.gob.es/kos/sector-publico/sector/vivienda""",2
"""http://datos.gob.es/kos/sector-publico/sector/ciencia-tecnologia""",2
"""http://datos.gob.es/kos/sector-publico/sector/comercio""",480
"""http://datos.gob.es/kos/sector-publico/sector/economia""",18
…,…
"""http://datos.gob.es/kos/sector-publico/sector/cultura-ocio""",212
"""http://datos.gob.es/kos/sector-publico/sector/sector-publico""",934
"""http://datos.gob.es/kos/sector-publico/sector/empleo""",14
